In [1]:
import flares as fl
import sunpy.map
import os
from sunpy.time import parse_time

import numpy as np
import data_prep as prep
import matplotlib.pyplot as plt

import warnings

## Prep 1

Cut out the flare region (approximately).

In [ ]:
path_aia = r'E:\Researches\2020EUVSolarFlare\Data\event20110215T01\1700A'
path_hmi = r'E:\Researches\2020EUVSolarFlare\Data\event20110215T01\Whitelight'

In [ ]:
def sec(day):
    return day * 24 * 3600
def timehmi(fits):
    return parse_time(fits[-36:-21])
def timeaia(fits):
    return parse_time(fits[-39:-22].replace('-',''))

In [ ]:
fits_list_aia = fl.get_fits_list(path_aia, 'aia')
fits_list_hmi = fl.get_fits_list(path_hmi, 'hmi')

fits_list_aia = prep.RemoveOverflowImage(fits_list_aia)

In [ ]:
# 列出匹配的aia, hmi图并合并为一个list
dataset_fits_list_aia = []
dataset_fits_list_hmi = []
for fits_hmi in fits_list_hmi:
    t1 = timehmi(fits_hmi)
    for fits_aia in fits_list_aia:
        t2 = timeaia(fits_aia)
        # print('Time:', t1, t2)
        dt = sec(t2 - t1)
        if abs(dt) < 30:
            print('Time Matched:', t1, t2, dt)
            dataset_fits_list_aia.append(fits_aia)
            dataset_fits_list_hmi.append(fits_hmi)
            fits_list_aia.remove(fits_aia) # 避免输出重复的data, 也避免不配对
            fits_list_hmi.remove(fits_hmi)
            break
        if dt > 60:
            # print('No match')
            break
fits_list = zip(dataset_fits_list_aia, dataset_fits_list_hmi)

In [ ]:
# 创建输出路径
path1 = os.path.join(path_aia, 'output')
path2 = os.path.join(path_hmi, 'output')
for path in [path1, path2]:
    if os.path.exists(path) is False:
        os.makedirs(path)

In [ ]:
for fits_aia, fits_hmi in fits_list:
    if 'lev1' in fits_aia and 'continuum' in fits_hmi:
        coord = fl.find_max_coord(fits_aia)
        fl.square_cut(sunpy.map.Map(fits_aia), coord, 1024, os.path.join(path_aia, 'output'))
        fl.square_cut(sunpy.map.Map(fits_hmi), coord, 1024, os.path.join(path_hmi, 'output'))

## Prep 2

resize the image. (not good)

In [ ]:
import astropy.units as u

In [ ]:
for fits_aia, fits_hmi in fits_list:
    if 'lev1' in fits_aia and 'continuum' in fits_hmi:
        coord = fl.find_max_coord(fits_aia)
        map_aia = fl.square_cut(sunpy.map.Map(fits_aia), coord, 256)
        new_dimensions = [256, 256] * u.pixel
        map_aia = map_aia.resample(new_dimensions)
        map_hmi = fl.square_cut(sunpy.map.Map(fits_hmi), coord, 256)
        break


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(projection=map_aia)
ax[1].plot(projection=map_hmi)
map_aia.plot(axes=ax[0])
map_hmi.plot(axes=ax[1])
plt.show()

## Prep 3

Delete those image with duplicate max points. (now in data_prep.py: RemoveOverflowImage)

In [ ]:
import data_prep as prep

In [ ]:
fits_list_aia = fl.get_fits_list(path1)
prep.RemoveOverflowImage(fits_list_aia)
fits_list_hmi = fl.get_fits_list(path2)
prep.RemoveOverflowImage(fits_list_hmi)

## Prep integrated with `data_prep`

Data preparation using data_prep. Currently in use.

List of flares close to center:
E:\Researches\2020EUVSolarFlare\Data\event20110215T01
E:\Researches\2020EUVSolarFlare\Data\event20110218T12(1 sunspot in middle)
E:\Researches\2020EUVSolarFlare\Data\event20110309T23
E:\Researches\2020EUVSolarFlare\Data\event20110314T19
E:\Researches\2020EUVSolarFlare\Data\event20110730T01
E:\Researches\2020EUVSolarFlare\Data\event20110906T21
E:\Researches\2020EUVSolarFlare\Data\event20111226T20
E:\Researches\2020EUVSolarFlare\Data\event20111231T12
E:\Researches\2020EUVSolarFlare\Data\event20120309T03(nice)
E:\Researches\2020EUVSolarFlare\Data\event20120510T04(nice)
E:\Researches\2020EUVSolarFlare\Data\event20120603T17
E:\Researches\2020EUVSolarFlare\Data\event20120704T09
E:\Researches\2020EUVSolarFlare\Data\event20120705T03
E:\Researches\2020EUVSolarFlare\Data\event20120705T11
E:\Researches\2020EUVSolarFlare\Data\event20120706T01
E:\Researches\2020EUVSolarFlare\Data\event20131110T04
E:\Researches\2020EUVSolarFlare\Data\event20140107T09
E:\Researches\2020EUVSolarFlare\Data\event20140329T17
E:\Researches\2020EUVSolarFlare\Data\event20141022T01
E:\Researches\2020EUVSolarFlare\Data\event20141022T13
E:\Researches\2020EUVSolarFlare\Data\event20141024T07

In [2]:
data_output_dir = r'E:\Program Files\VSCode\2021_ImagingWithDeepLearning\my_test_data_7'
# event_dir = r'E:\Researches\2020EUVSolarFlare\Data\event20110215T01' # 1st event
# event_dir = r'E:\Researches\2020EUVSolarFlare\Data\event20120705T03' # 2nd event
# event_dir = r'E:\Researches\2020EUVSolarFlare\Data\event20110309T23' # 3rd event
# event_dirs = [r'E:\Researches\2020EUVSolarFlare\Data_Lya\event20140930T18'] # lya vault

In [3]:
event_dirs = [
r'E:\Researches\2020EUVSolarFlare\Data\event20110215T01',
r'E:\Researches\2020EUVSolarFlare\Data\event20110218T12',
r'E:\Researches\2020EUVSolarFlare\Data\event20110309T23',
r'E:\Researches\2020EUVSolarFlare\Data\event20110314T19',
r'E:\Researches\2020EUVSolarFlare\Data\event20111226T20',
r'E:\Researches\2020EUVSolarFlare\Data\event20111231T12',
r'E:\Researches\2020EUVSolarFlare\Data\event20120309T03',
r'E:\Researches\2020EUVSolarFlare\Data\event20120510T04',
r'E:\Researches\2020EUVSolarFlare\Data\event20110730T01',# maximum outside of solar disk
r'E:\Researches\2020EUVSolarFlare\Data\event20110906T21',# maximum outside of solar disk
r'E:\Researches\2020EUVSolarFlare\Data\event20120603T17',# maximum outside of solar disk
r'E:\Researches\2020EUVSolarFlare\Data\event20120704T09',
r'E:\Researches\2020EUVSolarFlare\Data\event20120705T03',
r'E:\Researches\2020EUVSolarFlare\Data\event20120705T11',
r'E:\Researches\2020EUVSolarFlare\Data\event20120706T01',
r'E:\Researches\2020EUVSolarFlare\Data\event20131110T04',
r'E:\Researches\2020EUVSolarFlare\Data\event20140107T09',
r'E:\Researches\2020EUVSolarFlare\Data\event20140329T17',
]


In [4]:
warnings.filterwarnings("ignore")

In [5]:
for event_dir in event_dirs:
    dataset_fits_list = prep.GetMultiChannelData(event_dir, fl.WAVELENGTHS[1], target_wave='Whitelight')
    print((np.array(dataset_fits_list)).shape)

    dataset_fits_list = prep.OverflowDSRemove(dataset_fits_list)
    print((np.array(dataset_fits_list)).shape)

    # If none qualifies, skip.
    if len(dataset_fits_list) == 0:
        continue

    prep.SaveDSFitsList(dataset_fits_list, None, 256, data_output_dir, type='full_disk')



Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
(20, 10)
(13, 10)
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
Data matched: 10 channels
(16, 10)
(8, 10)
Data matched: 10 channels
Dat